In [ ]:
from jupyter_helpers import show, show_b, show_x, show_part, hide_objects_matching_strings

In [ ]:
import sys
sys.path.append("../src")

from cq_enclosure_builder import PartFactory as pf
from cq_enclosure_builder import Enclosure, EnclosureSize, Face, ProjectInfo
from cq_enclosure_builder import Panel, Face
from cq_enclosure_builder.layout_builder import LayoutElement, LayoutGroup

# 01 – Panel with buttons

In [ ]:
WIDTH     = 100
LENGTH    = 38
THICKNESS = 2

panel = Panel(Face.TOP, WIDTH, LENGTH, THICKNESS)

button = pf.build_button(
    part_type="SPST PBS-24B-4",
    enclosure_wall_thickness=THICKNESS
)

panel.add("SPST", button, rel_pos=(10, 0))
panel.add("SPST corner", button, abs_pos=(WIDTH, LENGTH))

panel.assemble()

show(panel.panel_with_debug)

# 02 – Set default part types and parameters

In [ ]:
pf.set_default_types({
    "button": 'SPST PBS-24B-4',
})
pf.set_default_parameters({
    "enclosure_wall_thickness": 2
})

panel = Panel(Face.TOP, 100, 38, 2)

button = pf.build_button()

panel.add("SPST", button, rel_pos=(-10, 0))
panel.add("SPST corner", button, abs_pos=(0, 0))

panel.assemble()

show(panel.panel_with_debug)

# 03 – Panel's optional parameters

In [ ]:
pf.set_default_types({"button": 'SPST PBS-24B-4'})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

panel = Panel(
    Face.TOP, 100, 38, 1.2,
    color=(1, 0, 0),  # colors are RGB with a scale from 0 to 1
    part_color=(0, 1, 0),
    alpha=0.2,  # affects the panel itself, not its parts
)

button = pf.build_button()

panel.add("SPST", pf.build_button(), rel_pos=(0, 5))

panel.assemble()

show(panel.panel_with_debug)

# 04 – Enclosure with buttons

In [ ]:
enclosure_size = EnclosureSize(180, 90, 38, 2)
enclosure = Enclosure(enclosure_size)

pf.set_default_types({"button": 'SPST PBS-24B-4'})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

enclosure.add_part_to_face(Face.TOP, "SPST", pf.build_button(), rel_pos=(-10, 0))

# The explosion factor and lid panel shift will helps you to see the insides of the enclosure
enclosure.assemble(walls_explosion_factor=1.5, lid_panel_shift=20)

show(enclosure.assembly)

# 05 – Export enclosure STLs

In [ ]:
enclosure_size = EnclosureSize(180, 90, 38, 2)
enclosure = Enclosure(enclosure_size)

pf.set_default_types({"button": 'SPST PBS-24B-4'})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

enclosure.add_part_to_face(Face.TOP, "SPST", pf.build_button(), abs_pos=(40, 10))

enclosure.assemble()

show(enclosure.assembly_with_debug)

# `enclosure.debug` (and `enclosure.assembly_with_debug`) will show
#   the 'printables', that can then be exported:

# Exports ready-to-print STLs in the 'stls/' folder
#   - my_project-box-v1.0.0.stl
#   - my_project-lid-v1.0.0.stl
# (names based on the default value of ProjectInfo; see next example)
enclosure.export_printables()

# 06 – Enclosure's optional parameters

In [ ]:
pf.set_default_types({"button": 'SPST PBS-24B-4'})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

enclosure = Enclosure(
    size=EnclosureSize(180, 90, 38, 2),
    project_info=ProjectInfo("my-enclosure", "0.9"),
    lid_on_faces=[Face.BOTTOM],    # for now, can only be BOTTOM, feel free to contribute/open an issue if you have a more specific need
    lid_panel_size_error_margin=1.2, # meaning the lid is `margin` smaller than the hole (space for the lid) on both width and length
    lid_thickness_error_margin=2,  # if >0, the lid screws and support will be slightly sunk in the enclosure
    add_corner_lid_screws=True,    # one screw per corner; more can be added with Enclosure#add_screw
    add_lid_support=True,     # adds a rim all around the enclosure to prevent the lid from sinking in
    add_top_support=True,     # small support rim on the enclosure to provide additional strength
    lid_screws_heat_set=True, # use heat set inserts instead of printed threads
    no_fillet_top=False,      # removed the fillet (rounded edges) on the top of the enclosure
    no_fillet_bottom=False,   # removed the fillet (rounded edges) on the bottom of the enclosure
)

enclosure.add_part_to_face(Face.TOP, "SPST", pf.build_button(), abs_pos=(40, 10))
enclosure.assemble()
show(enclosure.assembly)

# Exports files stls/my-enclosure-box-v0.9.stl and stls/my-enclosure-lid-v0.9.stl
enclosure.export_printables()

# 07 – Layout builder: simple line

In [ ]:
pf.set_default_types({
    "usb_a": '3.0 vertical cltgxdd',
    "button": 'SPST PBS-24B-4',
    "encoder": 'EC11',
})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

panel = Panel(Face.TOP, 100, 38, 2)

group = LayoutGroup.line_of_parts(
    [
        ("USB", pf.build_usb_a()),
        ("Button", pf.build_button()),
        ("Encoder", pf.build_encoder())
    ],
    margin=0,
    horizontal=True,
    group_center_at_0_0=False,
    align_start_to_outside_footprint=False,  # redundant if align_to_outside_footprint is True
    align_to_outside_footprint=False,
)

# Example of alignments, when group_center_at_0_0 is True:
# - if align_to_outside_footprint,
#   the left of the USB's outside footprint (i.e. the port's hole) will be at -X,
#   and the right of the Encoder's outside footprint (i.e. the cap) will be at +X
# - if not, it will use the inside footprint (i.e. the board)

# Example of alignments, when group_center_at_0_0 is False:
# - if align_[start_]to_outside_footprint, (0,0) will correspond to the the left of the USB's outside footprint
# - if not, it will use the inside footprint

# align_to_outside_footprint will make sure all the outside footprints touch each other (if margin=0)
# while align_start_to_outside_footprint will only make the first outside footprint start at (0,0) and use the total footprint for the rest

for idx, elem in enumerate(group.get_elements()):
    panel.add(elem.label, elem.part, rel_pos=elem.get_pos())

panel.assemble()

show(panel.panel_with_debug)

# 08 – Layout builder: fixed-width line

In [ ]:
pf.set_default_types({
    "usb_a": '3.0 vertical cltgxdd',
    "button": 'SPST PBS-24B-4',
    "encoder": 'EC11',
})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

WIDTH = 100
LENGTH = 38

panel = Panel(Face.TOP, WIDTH, LENGTH, 2)

# fixed_width_line_of_parts will spread the parts so the first is directly on
#   the starting at 0, and the last ending at WIDTH

# If using align_to_outside_footprint,
#   0 and WIDTH will be aligned to the outside footprint of the parts
# If using add_margin_on_sides, the line will start and end
#   at the same distance from the borders as the distance between each part

group = LayoutGroup.fixed_width_line_of_parts(
    WIDTH,
    [
        ("USB", pf.build_usb_a()),
        ("Button", pf.build_button()),
        ("Encoder", pf.build_encoder())
    ],
    horizontal=True,
    add_margin_on_sides=False,
    group_center_at_0_0=False,
    elements_centers_at_0_0=True,
    align_to_outside_footprint=True,
)

group.translate([0, LENGTH/2, 0])

for idx, elem in enumerate(group.get_elements()):
    panel.add(elem.label, elem.part, abs_pos=elem.get_pos())

panel.assemble()

show(panel.panel_with_debug)

# 09 – Layout builder: fixed-width line with median part centred at 0,0

In [ ]:
pf.set_default_types({
    "usb_a": '3.0 vertical cltgxdd',
    "button": 'SPST PBS-24B-4',
    "encoder": 'EC11',
})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

WIDTH = 100
LENGTH = 38

panel = Panel(Face.TOP, WIDTH, LENGTH, 2)

button_element = LayoutElement("Button", pf.build_button())
usb_element = LayoutElement("USB", pf.build_usb_a())
encoder_element = LayoutElement("Encoder", pf.build_encoder())
button_element.set_footprints_x(encoder_element.total_footprint[0])

# A hacky way to keep the middle element's center at (0,0) is to override the footprint
#   of the element[s] on its left and the element[s] of its right to be equal
# The margins, of course, won't be equal, but it can work better for some layout
#   (see Octopus' front panel for example: we want the screen at 0,0)

group = LayoutGroup.fixed_width_line_of_elements(
    WIDTH,
    [button_element, usb_element, encoder_element],
    horizontal=True,
    add_margin_on_sides=True,
    group_center_at_0_0=False,
    elements_centers_at_0_0=True,
    align_to_outside_footprint=True,
)

group.translate([0, LENGTH/2, 0])
for idx, elem in enumerate(group.get_elements()):
    panel.add(elem.label, elem.part, abs_pos=elem.get_pos())
panel.assemble()
show(panel.panel_with_debug)

# 10 – Layout builder: grid of parts

In [ ]:
pf.set_default_types({"jack": '6.35mm PJ-612A'})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

panel = Panel(Face.TOP, 100, 38, 2)

jacks_grid = LayoutGroup.grid_of_part(
    "Jack 6.35",
    pf.build_jack(),
    rows=2,
    cols=5,
    margin_rows=0,
    margin_cols=2,
)

for idx, elem in enumerate(jacks_grid.get_elements()):
    panel.add(elem.label, elem.part, rel_pos=elem.get_pos())
panel.assemble()
show(panel.panel_with_debug)

# 11 – Layout builder: combining groups

In [ ]:
pf.set_default_parameters({"enclosure_wall_thickness": 2})

panel = Panel(Face.TOP, 200, 80, 2)

jacks_grid_1 = LayoutGroup.grid_of_part("Jack 6.35", pf.build_jack(part_type="6.35mm PJ-612A"), rows=2, cols=5, margin_rows=5, margin_cols=2)
jacks_grid_2 = LayoutGroup.grid_of_part("Jack 3.5", pf.build_jack(part_type="3.5mm XXX"), rows=4, cols=3, margin_rows=0, margin_cols=0)
one_usb = LayoutElement("USB", pf.build_usb_a(part_type="3.0 vertical cltgxdd"))

all_jacks = LayoutGroup.fixed_width_line_of_elements(
    200,
    [jacks_grid_1, jacks_grid_2, one_usb],
    align_to_outside_footprint=True,
)

for idx, elem in enumerate(all_jacks.get_elements()):
    panel.add(elem.label, elem.part, rel_pos=elem.get_pos())
panel.assemble()
show(panel.panel_with_debug)

# 12 – Text

In [ ]:
enclosure_size = EnclosureSize(180, 90, 38, 2)
enclosure = Enclosure(enclosure_size)

pf.set_default_types({"text": 'default'})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

extruded_text = pf.build_text(
    text = "Sample text\nA new line",
)

cut_text = pf.build_text(
    text = "Hello",
    thickness = 5.6,
    cut = True,
    fontsize = 8,
    width = 30,
    length = 9,
    outside = True,
)

enclosure.add_part_to_face(Face.TOP, "Extruded", extruded_text, rel_pos=(0, -6))
enclosure.add_part_to_face(Face.TOP, "Cut", cut_text, rel_pos=(0, 10))

enclosure.assemble()

show(enclosure.assembly, hide_contains=["BOTTOM", "FRONT", "RIGHT"])

# 13 – Support for parts

In [ ]:
enclosure_size = EnclosureSize(180, 90, 38, 2)
enclosure = Enclosure(enclosure_size)

pf.set_default_types({
    "button": 'SPST PBS-24B-4',
    "support": 'pyramid',
})
pf.set_default_parameters({"enclosure_wall_thickness": 2})

spst = pf.build_button()

support_height = enclosure_size.outer_thickness - spst.inside_footprint_thickness - enclosure_size.wall_thickness
support = pf.build_support(support_height=support_height)

enclosure.add_part_to_face(Face.TOP, "SPST", spst, rel_pos=(-10, 0))
enclosure.add_part_to_face(Face.BOTTOM, "Support for SPST", support, rel_pos=(-10, 0))

enclosure.assemble()

# To view how, inside the enclosure, a pillar is supporting the button from underneath,
#   you'll want to enable the wireframe mode (or use Jupyter-CadQuery for more control)
show(enclosure.assembly_with_debug, hide_contains=["Printables", "FRONT", "RIGHT"])

# 14 – All parts

In [ ]:
pf.set_default_parameters({
    "enclosure_wall_thickness": 2,
    "support_height": 10,  # used by support/pyramid
    "width": 6,
    "length": 5,
})
# Will break if new Parts with arguments without default values are added

panel = Panel(Face.TOP, 0.1, 0.1, 0.1)  # not caring about the panel size

lines = []

for category in pf.list_categories():
    part_types = pf.list_types_for_category(category)
    parts = []
    for part_type in part_types:
        parts.append((part_type, pf.build(category=category, part_type=part_type)))

    group = LayoutGroup.line_of_parts(
        parts,
        margin=5,
        horizontal=True,
        group_center_at_0_0=True,
    )
    lines.append(group)

group_of_groups = LayoutGroup.line_of_elements(
    lines,
    margin=15,
    horizontal=False,
    group_center_at_0_0=True,
)

for idx, elem in enumerate(group_of_groups.get_elements()):
    panel.add(elem.label, elem.part, rel_pos=elem.get_pos())

panel.assemble()

show(panel.panel_with_debug)

# Strength test enclosures

In [ ]:
import sys
sys.path.append("../src")

from cq_enclosure_builder import PartFactory as pf
from cq_enclosure_builder import Enclosure, EnclosureSize, Face, ProjectInfo
from cq_enclosure_builder import Panel, Face
from cq_enclosure_builder.layout_builder import LayoutElement, LayoutGroup
from cq_enclosure_builder.parts.common.knobs_and_caps import KNOB_18_x_17_25

pf.set_default_types({
    "button": 'SPST PBS-24B-4',
    "potentiometer": 'WH148',
    "barrel_plug": 'DC-022B',
    "jack": '6.35mm PJ-612A',
    "support": 'pyramid',
})
pf.set_default_parameters({
    "enclosure_wall_thickness": 2,
    "pot_knob": KNOB_18_x_17_25,
})

def build_strength_test_enclosure(with_support: bool):
    enclosure_size = EnclosureSize(60, 113, 31, 2)  # 1590B
    project_name = "strength-test-" + ("with" if with_support else "without") + "-support"
    project_info = ProjectInfo(project_name, "1")
    enclosure = Enclosure(enclosure_size, project_info=project_info)

    pots = LayoutGroup.fixed_width_line_of_parts(
        enclosure_size.outer_width,
        [
            ("Pot left", pf.build_potentiometer()),
            ("Pot right", pf.build_potentiometer()),
        ],
        add_margin_on_sides=True,
        group_center_at_0_0=True,
        elements_centers_at_0_0=True,
        align_to_outside_footprint=False,
    )
    pots.translate([0, 30, 0])
    for idx, elem in enumerate(pots.get_elements()):
        enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, rel_pos=elem.get_pos())

    spst = pf.build_button()
    enclosure.add_part_to_face(Face.TOP, "SPST", spst, rel_pos=(0, -30))

    if with_support:
        support_height = enclosure_size.outer_thickness - spst.inside_footprint_thickness - enclosure_size.wall_thickness
        support = pf.build_support(support_height=support_height)

        enclosure.add_part_to_face(Face.BOTTOM, "Support for SPST", support, rel_pos=(0, 30))

    enclosure.add_part_to_face(Face.BACK, "Barrel plug", pf.build_barrel_plug(), rel_pos=(0, -5))

    enclosure.add_part_to_face(Face.LEFT, "Jack out", pf.build_jack(), rel_pos=(0, 0))
    enclosure.add_part_to_face(Face.RIGHT, "Jack in", pf.build_jack(), rel_pos=(0, 0))

    enclosure.assemble()

    return enclosure

without_support = build_strength_test_enclosure(False)
with_support = build_strength_test_enclosure(True)

show(without_support.assembly_with_debug, hide_contains=["Panels masks", "Printables", "FRONT", "RIGHT"])
show(with_support.assembly_with_debug, hide_contains=["Panels masks", "Printables", "FRONT", "RIGHT"])

without_support.export_printables()
with_support.export_printables()